In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation

In [3]:
# Model definition
# Add graph
lung_model = BayesianNetwork([('Inflammation', 'Wellness'), ('Inflammation', 'Bacterial load'), ('Bacterial load', 'FEV1')])

# Define the CPDs
cpd_i = TabularCPD(variable='Inflammation', variable_card=3, values=[[0.3], [0.5], [0.2]])
cpd_w = TabularCPD(variable='Wellness', variable_card=5,
                      values=[[0.01, 0.2, 0.4], [0.09, 0.3, 0.3], [0.2, 0.3, 0.15], [0.3, 0.1, 0.1], [0.4, 0.1, 0.05]],
                      evidence=['Inflammation'], evidence_card=[3])
cpd_bl = TabularCPD(variable='Bacterial load', variable_card=3,
                    values=[[0.6, 0.3, 0.1], [0.2, 0.3, 0.3], [0.2, 0.4, 0.6]],
                    evidence=['Inflammation'], evidence_card=[3])
cpd_FEV1 = TabularCPD(variable='FEV1', variable_card=3, values=[[0.6, 0.3, 0.1], [0.3, 0.4, 0.4], [0.1, 0.3, 0.5]],
                   evidence=['Bacterial load'],
                   evidence_card=[3]
                   )
# Add the CPDs to the model
lung_model.add_cpds(cpd_i, cpd_w, cpd_bl, cpd_FEV1)

# Checks
# Show CPDs
print("CPDs defined:", lung_model.get_cpds())
# Checking if the CPDs are valid for the model
print("\nThe CPDs are valid:", lung_model.check_model())
# List of independence conditions for Bacterial load
print("\nIndependence conditions for Bacterial load:", lung_model.local_independencies('Bacterial load'))
# Get all model independence conditions
print("\nIndependence conditions across the whole model:\n", lung_model.get_independencies())

CPDs defined: [<TabularCPD representing P(Inflammation:3) at 0x7fa73376c070>, <TabularCPD representing P(Wellness:5 | Inflammation:3) at 0x7fa73376c0a0>, <TabularCPD representing P(Bacterial load:3 | Inflammation:3) at 0x7fa73376c100>, <TabularCPD representing P(FEV1:3 | Bacterial load:3) at 0x7fa73376d000>]

The CPDs are valid: True

Independence conditions for Bacterial load: (Bacterial load ⟂ Wellness | Inflammation)

Independence conditions across the whole model:
 (FEV1 ⟂ Wellness | Inflammation)
(FEV1 ⟂ Inflammation, Wellness | Bacterial load)
(FEV1 ⟂ Wellness | Inflammation, Bacterial load)
(FEV1 ⟂ Inflammation | Wellness, Bacterial load)
(Inflammation ⟂ FEV1 | Bacterial load)
(Inflammation ⟂ FEV1 | Wellness, Bacterial load)
(Bacterial load ⟂ Wellness | Inflammation)
(Bacterial load ⟂ Wellness | FEV1, Inflammation)
(Wellness ⟂ FEV1, Bacterial load | Inflammation)
(Wellness ⟂ FEV1 | Bacterial load)
(Wellness ⟂ Bacterial load | FEV1, Inflammation)
(Wellness ⟂ FEV1 | Inflammation, 

In [4]:
# CASE 1
belief_propagation = BeliefPropagation(lung_model)
final_dist = belief_propagation.query(variables=['Inflammation'], evidence={'Bacterial load': 1})
print(final_dist.values)

  0%|          | 0/2 [00:00<?, ?it/s]

[0.22222222 0.55555556 0.22222222]


In [8]:
belief_propagation = BeliefPropagation(lung_model)
final_dist = belief_propagation.query(variables=['Inflammation'], evidence={'FEV1': 0, 'Wellness': 2})
print(final_dist.values)

  0%|          | 0/1 [00:00<?, ?it/s]

[0.33333333 0.58712121 0.07954545]


In [14]:
belief_propagation = BeliefPropagation(lung_model)
final_dist = belief_propagation.query(variables=['FEV1'], evidence={'Wellness': 0})
print(final_dist.values)

  0%|          | 0/2 [00:00<?, ?it/s]

[0.2684153  0.37825137 0.35333333]
